<h3>0.0 CARGA DE BIBLIOTECAS, VARIABLES, CLASES Y MÉTODOS IMPORTANTES</h3>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, LSTM, Dense, Embedding
import pickle
import json
import os
import sys

num_lilith = '05'
archivo_json_original = '../../Chessia_Data_Convert/Target Data/original/Datos-V2.json'
archivo_json_expansion = '../../Chessia_Data_Convert/Target Data/expansion/Datos-V2.json'
tokens_lilith = "../../Chessia_Data_Convert/TokensGenerator/general_tokens_v1.pkl"

class Partida:
    def __init__(self):
        self.movimientos = []
        self.resultado = None

# Función para cargar partidas en lotes
def cargar_partidas_por_rango(archivo_json_original, inicio=0, fin=None):
    total_movimientos = []
    total_resultados = []
    with open(archivo_json_original, 'r') as file:
        # Carga el contenido completo como una lista de objetos JSON
        lista_json = json.load(file)
        
        if fin is None or fin > len(lista_json):
            fin = len(lista_json)

        # Itera sobre cada objeto JSON en la lista
        for i in range(inicio, fin):
            total_movimientos.append(lista_json[i]['partida']['movimientos'])
            total_resultados.append(lista_json[i]['partida']['resultado'])
            
    return total_movimientos, total_resultados

def ver_tablas(history):
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))

    epocas = range(1, len(history.history['loss']) + 1)

    # Graficar la precisión
    axs[0].plot(epocas, history.history['accuracy'], 'r', label='Precisión en entrenamiento')
    axs[0].plot(epocas, history.history['val_accuracy'], 'b', label='Precisión en validación')
    axs[0].set_title('Precisión en entrenamiento y validación')
    axs[0].set_xlabel('Épocas')
    axs[0].set_ylabel('Precisión')
    axs[0].legend()

    # Graficar la pérdida
    axs[1].plot(epocas, history.history['loss'], 'r', label='Pérdida en entrenamiento')
    axs[1].plot(epocas, history.history['val_loss'], 'b', label='Pérdida en validación')
    axs[1].set_title('Pérdida en entrenamiento y validación')
    axs[1].set_xlabel('Épocas')
    axs[1].set_ylabel('Pérdida')
    axs[1].legend()

    plt.tight_layout()  # Ajustar el diseño para evitar superposiciones
    plt.show()


class PlotMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        try:
            if self.model.history:
                # ver_tablas(self.model.history)
                pass

        except Exception as e:
            print("Error al mostrar la gráfica:", e)

<h3>3.0 PROCESADO DE DATOS DE LILITH - USA ProcesadorPGN_V2 Y EXTENSIÓN DE PERTIDAS</h3>

In [2]:
# Convertir el generador en una lista y extraer la primera tupla
recogido = cargar_partidas_por_rango(archivo_json_original, fin=100000)
recogido += cargar_partidas_por_rango(archivo_json_expansion)

movimientos = recogido[0]
resultados = recogido[1]

# Crear y ajustar el tokenizador
tokenizer = Tokenizer()
with open(tokens_lilith, "rb") as f:
    tokenizer = pickle.load(f)

# Convertir todas las jugadas a secuencias de números enteros y luego a tensores válidos
movimientos_tokenized = tokenizer.texts_to_sequences(movimientos)
movimientos_tokenized = pad_sequences(movimientos_tokenized, maxlen=500, padding="pre")

resultados_tokenized = tf.one_hot([int(resultado) for resultado in resultados], depth=3)  # 3 clases: empate, victoria blanca, victoria negra

tensor_mov_entrena = tf.constant(movimientos_tokenized[:int(len(movimientos_tokenized) * 0.8) ])
tensor_res_entrena = tf.constant(resultados_tokenized [:int(len(resultados_tokenized)  * 0.8) ])
tensor_mov_prueba =  tf.constant(movimientos_tokenized[ int(len(movimientos_tokenized) * 0.8):])
tensor_res_prueba =  tf.constant(resultados_tokenized [ int(len(resultados_tokenized)  * 0.8):])

<h3>3.1 LILITH - DEFINICIÓN, ENTRENAMIENTO Y GRÁFICAS DE DATOS CON POCOS VALORES</h3>

<h4>&ensp;&ensp;3.1.1 CREACIÓN DEL MODELO</h4>

In [3]:
# Definir el modelo
modelo = tf.keras.Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128),
    LSTM(units=256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(units=128, dropout=0.2, recurrent_dropout=0.2),
    # Dense(units=256, activation='relu'),
    # Dropout(0.3),
    Dense(units=3, activation='softmax')
])

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
modelo.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

<h4>&ensp;&ensp;3.1.2 ENTRENAMIENTO</h4>

In [5]:
history = None
try:
    # Definir callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint(filepath='modelo_checkpoint.keras', save_best_only=True)
    plot_metrics = PlotMetricsCallback()

    try:
        modelo = tf.keras.models.load_model('modelo_checkpoint.keras')
        print("Checkpoint cargado. Continuando entrenamiento desde el último punto guardado...\n\n")
    except:
        print("No se encontró un checkpoint existente. Iniciando un nuevo entrenamiento...\n\n")

    # Entrenar el modelo con los callbacks
    history = modelo.fit(
        tensor_mov_entrena,
        tensor_res_entrena,
        epochs=20,
        initial_epoch=8,  # Especificar la primera época a ejecutar
        batch_size=256,
        validation_data=(tensor_mov_prueba, tensor_res_prueba),
        callbacks=[early_stopping, checkpoint, plot_metrics]
        # callbacks=[early_stopping, checkpoint]
    )

    # Guardar el modelo final
    modelo.save("lilith"+num_lilith+".keras")

    # Evaluar el modelo en los datos de prueba
    loss, accuracy = modelo.evaluate(tensor_mov_prueba, tensor_res_prueba)
    print("\n\nLoss en los datos de prueba:", loss)
    print("Precisión en los datos de prueba:", accuracy)

except:
    if history is not None:
        ver_tablas(history)
    else:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print(f"\nError: {exc_type.__name__}: {exc_value}\n")

Checkpoint cargado. Continuando entrenamiento desde el último punto guardado...


Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1164s 4s/step - accuracy: 0.6827 - loss: 0.7340 - val_accuracy: 0.6518 - val_loss: 0.7934
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1282s 4s/step - accuracy: 0.7004 - loss: 0.7062 - val_accuracy: 0.6545 - val_loss: 0.7963
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1380s 4s/step - accuracy: 0.7058 - loss: 0.6926 - val_accuracy: 0.6598 - val_loss: 0.7900
Epoch 12/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1458s 5s/step - accuracy: 0.7143 - loss: 0.6744 - val_accuracy: 0.6659 - val_loss: 0.7944
Epoch 13/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1512s 5s/step - accuracy: 0.7252 - loss: 0.6527 - val_accuracy: 0.6566 - val_loss: 0.8063
Epoch 14/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1565s 5s/step - accuracy: 0.7305 - loss: 0.6388 - val_accuracy: 0.6572 - val_loss: 0.8050
Epoch 15/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1629s 5s/step - accuracy: 0.7430 - loss: 0.6138 - val_accuracy: 0.6587 - val_loss: 0.8083
Ep

<h4>&ensp;&ensp;3.1.3 ESTADÍSTICAS Y GRÁFICAS</h4>

In [ ]:
ver_tablas(history)

<h3>3.2 PRUEBA DE PREDICCIÓN LILITH</h3>

<h4>&ensp;&ensp;3.2.1 RECOGIDA DE DATOS Y TOKENS</h4>

In [3]:
movimientos, resultados = cargar_partidas_por_rango(archivo_json_original, inicio=100000)

tokenizer = Tokenizer()
with open(tokens_lilith, "rb") as f:
    tokenizer = pickle.load(f)

tensor_mov_pred = tf.constant(pad_sequences(tokenizer.texts_to_sequences(movimientos), maxlen=500, padding="pre"))

<h4>&ensp;&ensp;3.2.2 PREDICCIÓN Y MUESTRA DE DATOS</h4>

In [4]:
np.set_printoptions(suppress=True, precision=4)

lilith = tf.keras.models.load_model("lilith"+num_lilith+".keras")

res_pred = lilith.predict(tensor_mov_pred)
aciertos = sumaAciertos = 0

for i in range(len(res_pred)):
    if int(resultados[i]) != int(np.argmax(res_pred[i])): char = '❌'
    else: 
        char = '✅'
        aciertos += 1
        sumaAciertos += res_pred[i][np.argmax(res_pred[i])]

    print(f"Predicción: {str(res_pred[i]):>22} -- Dice: {np.argmax(res_pred[i])} -- Real: {resultados[i]}  {char}  --- Aciertos: {aciertos:>5}/{i+1:<5} - {int(aciertos/(i+1)*100):>3}%")


105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step
Predicción: [0.3801 0.4622 0.1577] -- Dice: 1 -- Real: 0  ❌  --- Aciertos:     0/1     -   0%
Predicción: [0.8899 0.0816 0.0285] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     0/2     -   0%
Predicción: [0.0307 0.0368 0.9325] -- Dice: 2 -- Real: 1  ❌  --- Aciertos:     0/3     -   0%
Predicción: [0.0355 0.8104 0.154 ] -- Dice: 1 -- Real: 1  ✅  --- Aciertos:     1/4     -  25%
Predicción: [0.5609 0.3717 0.0674] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     1/5     -  20%
Predicción: [0.0621 0.8266 0.1114] -- Dice: 1 -- Real: 1  ✅  --- Aciertos:     2/6     -  33%
Predicción: [0.4468 0.1749 0.3782] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     2/7     -  28%
Predicción: [0.2353 0.4614 0.3033] -- Dice: 1 -- Real: 2  ❌  --- Aciertos:     2/8     -  25%
Predicción: [0.1369 0.5063 0.3568] -- Dice: 1 -- Real: 0  ❌  --- Aciertos:     2/9     -  22%
Predicción: [0.3173 0.4166 0.2661] -- Dice: 1 -- Real: 0  ❌  --- Aciertos:     2/10    -  20%
Predicción: [0.3

In [17]:
print(f"{sumaAciertos/len(res_pred)}")

0.6591934644464237
